In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [49]:
!wget "https://download.microsoft.com/download/b/8/8/b88c0bae-e6c1-43e1-8726-98cf5af36ca4/ASL_Citizen.zip" \
-O /content/drive/MyDrive/ASL_Citizen.zip


--2025-12-13 18:05:47--  https://download.microsoft.com/download/b/8/8/b88c0bae-e6c1-43e1-8726-98cf5af36ca4/ASL_Citizen.zip
Resolving download.microsoft.com (download.microsoft.com)... 104.119.104.215, 2600:1413:5000:12ae::317f, 2600:1413:5000:12a1::317f
Connecting to download.microsoft.com (download.microsoft.com)|104.119.104.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45924134223 (43G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/ASL_Citizen.zip’

/content/drive/MyDr  71%[=============>      ]  30.67G  51.0MB/s    in 10m 6s  


Cannot write to ‘/content/drive/MyDrive/ASL_Citizen.zip’ (Success).


In [9]:
!sync
!sleep 20
!sync



In [47]:
ls -lh /content/drive/MyDrive/ASL_Citizen.zip

-rw------- 1 root root 3.1G Dec 13 18:03 /content/drive/MyDrive/ASL_Citizen.zip


In [1]:
!ls -lh /content/drive/MyDrive/

ls: cannot access '/content/drive/MyDrive/': No such file or directory


In [55]:
zip_path = "/content/drive/MyDrive/ASL_Citizen.zip"

In [56]:
import zipfile

zip_path = "/content/drive/MyDrive/ASL_Citizen.zip"

with zipfile.ZipFile(zip_path, 'r') as z:
    files = z.namelist()

len(files), files[:10]

BadZipFile: File is not a zip file

In [14]:
import zipfile
import numpy as np
import cv2
import tempfile
import os

zip_path = "/content/drive/MyDrive/ASL_Citizen.zip"
video_file_in_zip = "ASL_Citizen/videos/2905202476287574-FAR.mp4"

with tempfile.TemporaryDirectory() as tmpdir:
    extracted_video_path = os.path.join(tmpdir, os.path.basename(video_file_in_zip))

    with zipfile.ZipFile(zip_path, 'r') as z:
        with z.open(video_file_in_zip) as source, open(extracted_video_path, 'wb') as target:
            target.write(source.read())

    cap = cv2.VideoCapture(extracted_video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {extracted_video_path}")
        vid = None
    else:
        ret, frame = cap.read()
        if ret:
            vid = frame
        else:
            print("Error: Could not read first frame.")
            vid = None
        cap.release()

if vid is not None:
    print(vid.shape)
else:
    print("Could not process video file.")

(540, 960, 3)


In [15]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 23.3 MB/s eta 0:00:00


In [16]:
import av
import zipfile
import numpy as np

class ZipVideoLoader:
    def __init__(self, zip_path):
        self.z = zipfile.ZipFile(zip_path, 'r')

    def read_video(self, video_path, target_frames=16):
        data = self.z.read(video_path)

        container = av.open(data)

        frames = []
        for frame in container.decode(video=0):
            img = frame.to_ndarray(format="rgb24")
            frames.append(img)

        if len(frames) >= target_frames:
            idx = np.linspace(0, len(frames)-1, target_frames).astype(int)
            frames = [frames[i] for i in idx]
        else:
            while len(frames) < target_frames:
                frames.append(frames[-1])

        return np.array(frames)


In [39]:
import zipfile
import numpy as np
import av
import io
import cv2

class ZipVideoLoader:
    def __init__(self, zip_path, resize=(224, 224)):
        self.z = zipfile.ZipFile(zip_path, 'r')
        self.resize = resize

    def read_video(self, video_path, target_frames=16):
        data = self.z.read(video_path)

        container = av.open(io.BytesIO(data))

        frames = []
        for frame in container.decode(video=0):
            img = frame.to_ndarray(format="rgb24")
            img = cv2.resize(img, self.resize, interpolation=cv2.INTER_AREA)
            frames.append(img)

        if len(frames) >= target_frames:
            idx = np.linspace(0, len(frames)-1, target_frames).astype(int)
            frames = [frames[i] for i in idx]
        else:
            while len(frames) < target_frames:
                frames.append(frames[-1])

        return np.array(frames)

zip_path = "/content/drive/MyDrive/ASL_Citizen.zip"
video_file = "ASL_Citizen/videos/2905202476287574-FAR.mp4"

loader = ZipVideoLoader(zip_path, resize=(224,224))
frames = loader.read_video(video_file, target_frames=16)

print(frames.shape)

(16, 224, 224, 3)


In [ ]:
!ls /content/drive/MyDrive

In [18]:
!unzip -l /content/drive/MyDrive/ASL_Citizen.zip | head -n 40

Archive:  /content/drive/MyDrive/ASL_Citizen.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2023-05-12 14:39   ASL_Citizen/
        0  2023-05-12 14:31   ASL_Citizen/splits/
   510233  2023-05-12 14:31   ASL_Citizen/splits/val.csv
  1998546  2023-05-12 14:31   ASL_Citizen/splits/train.csv
  1625987  2023-05-12 14:31   ASL_Citizen/splits/test.csv
        0  2023-05-10 22:45   ASL_Citizen/videos/
   460924  2023-05-10 22:45   ASL_Citizen/videos/5086723486875651-RUN.mp4
   773631  2023-05-10 22:45   ASL_Citizen/videos/2233415645150214-SWEET.mp4
   307814  2023-05-10 22:44   ASL_Citizen/videos/09412352698319548-POSSIBLE.mp4
   697942  2023-05-10 22:44   ASL_Citizen/videos/6184880487426632-PAINT.mp4
   560630  2023-05-10 22:44   ASL_Citizen/videos/6682045893304822-RACE.mp4
   594852  2023-05-10 22:44   ASL_Citizen/videos/22425009794174322-SCULPT.mp4
  1129896  2023-05-10 22:44   ASL_Citizen/videos/8563900568328626-SCAR MEMORY.mp4
   504519  2023-05-10 2

In [19]:
import pandas as pd
import zipfile
import io

zip_path = "/content/drive/MyDrive/ASL_Citizen.zip"

def read_csv_from_zip(zip_path, csv_name):
    with zipfile.ZipFile(zip_path) as z:
        with z.open(csv_name) as f:
            return pd.read_csv(f)

In [20]:
train_df = read_csv_from_zip(zip_path, "ASL_Citizen/splits/train.csv")
val_df   = read_csv_from_zip(zip_path, "ASL_Citizen/splits/val.csv")
test_df  = read_csv_from_zip(zip_path, "ASL_Citizen/splits/test.csv")

train_df.head()

,Participant ID,Video file,Gloss,ASL-LEX Code
0,P1,15890366051589533-APPLE.mp4,APPLE,A_03_054
1,P1,35618482303951104-IMPOSSIBLE.mp4,IMPOSSIBLE,B_01_032
2,P1,6958143575951994-PARK.mp4,PARK,E_03_028
3,P1,8006032738002744-SOCCER 2.mp4,SOCCER2,F_03_032
4,P1,37542279833186454-STINK.mp4,STINK,H_01_064


In [33]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

def add_paths(df): # Redefine add_paths as it was overwritten later
    df["video_path"] = "ASL_Citizen/videos/" + df["Video file"]
    return df

train_df = add_paths(train_df)
val_df = add_paths(val_df)
test_df = add_paths(test_df)

# Encode 'Gloss' into numerical 'label' for the dataset
label_encoder = LabelEncoder()
label_encoder.fit(train_df['Gloss'])

train_df['label'] = label_encoder.transform(train_df['Gloss'])
val_df['label'] = label_encoder.transform(val_df['Gloss'])
# num_classes was already defined using train_df['Gloss'].nunique() which is equivalent


In [22]:
def add_paths(df):
    df["video_path"] = "ASL_Citizen/videos/" + df["Video file"]
    return df

train_df = add_paths(train_df)
val_df = add_paths(val_df)
test_df = add_paths(test_df)

In [53]:
class ASLCitizenDataset(Dataset):
    def __init__(self, df, zip_path, frames=16, transform=None):
        self.df = df.reset_index(drop=True)
        self.frames = frames
        self.transform = transform
        self.loader = ZipVideoLoader(zip_path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        video_path = row["video_path"]
        label = int(row["label"])

        try:
            frames = self.loader.read_video(
                video_path,
                target_frames=self.frames
            )

            frames = torch.tensor(frames).permute(0, 3, 1, 2).float() / 255.
            return frames, label

        except Exception as e:
            print(f"[WARN] Skipping corrupted video: {video_path}")
            new_idx = torch.randint(0, len(self.df), (1,)).item()
            return self.__getitem__(new_idx)


In [54]:
from torch.utils.data import DataLoader

train_ds = ASLCitizenDataset(train_df, zip_path, resize=(224,224))
val_ds   = ASLCitizenDataset(val_df, zip_path, resize=(224,224))

train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=2)

BadZipFile: File is not a zip file

# **train optical flow model**

In [ ]:
import zipfile, pandas as pd

ZIP_PATH = "/content/drive/MyDrive/ASL_Citizen.zip"

def read_csv_from_zip(zip_path, inner_csv_path):
    with zipfile.ZipFile(zip_path) as z:
        with z.open(inner_csv_path) as f:
            return pd.read_csv(f)

train_df = read_csv_from_zip(ZIP_PATH, "ASL_Citizen/splits/train.csv")
val_df   = read_csv_from_zip(ZIP_PATH, "ASL_Citizen/splits/val.csv")

print(train_df.columns)
print(train_df.head(2))


Index(['Participant ID', 'Video file', 'Gloss', 'ASL-LEX Code'], dtype='object')
  Participant ID                        Video file       Gloss ASL-LEX Code
0             P1       15890366051589533-APPLE.mp4       APPLE     A_03_054
1             P1  35618482303951104-IMPOSSIBLE.mp4  IMPOSSIBLE     B_01_032


In [ ]:
import av, io, numpy as np, cv2, torch
from torch.utils.data import Dataset

class ZipVideoLoader:
    def __init__(self, zip_path):
        self.zip_path = zip_path
        self.z = zipfile.ZipFile(zip_path, 'r')

    def read_frames(self, video_path, target_frames=16, resize=(224,224)):
        data = self.z.read(video_path)
        container = av.open(io.BytesIO(data))
        frames = []
        for frame in container.decode(video=0):
            img = frame.to_ndarray(format="rgb24")
            frames.append(img)

        if len(frames) == 0:
            raise RuntimeError(f"No frames decoded: {video_path}")

        if len(frames) >= target_frames:
            idx = np.linspace(0, len(frames)-1, target_frames).astype(int)
            frames = [frames[i] for i in idx]
        else:
            while len(frames) < target_frames:
                frames.append(frames[-1])

        out = []
        for fr in frames:
            fr = cv2.resize(fr, resize, interpolation=cv2.INTER_AREA)
            out.append(fr)
        return np.stack(out, axis=0)


def compute_flow_farneback(frames_rgb):
    """
    frames_rgb: (T,H,W,3) uint8 RGB
    return: flow tensor (2, T-1, H, W) float32
    """
    T, H, W, _ = frames_rgb.shape
    prev = cv2.cvtColor(frames_rgb[0], cv2.COLOR_RGB2GRAY)
    flows = []
    for t in range(1, T):
        nxt = cv2.cvtColor(frames_rgb[t], cv2.COLOR_RGB2GRAY)
        flow = cv2.calcOpticalFlowFarneback(
            prev, nxt, None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )
        flows.append(flow)
        prev = nxt

    flow = np.stack(flows, axis=0)
    flow = np.transpose(flow, (3,0,1,2))

    flow = np.clip(flow, -20.0, 20.0) / 20.0
    return flow.astype(np.float32)


In [ ]:
import torch
from torch.utils.data import Dataset
import av, io, numpy as np, cv2

class ASLCitizenFlowDataset(Dataset):
    def __init__(self, df, zip_path, frames=16, resize=(224,224), use_cache=True):
        self.df = df.reset_index(drop=True)
        self.frames = frames
        self.resize = resize
        self.use_cache = use_cache

        self.z = zipfile.ZipFile(zip_path, 'r')
        self.cache = {}   # Cache

    def __len__(self):
        return len(self.df)

    def read_frames(self, video_path):
        data = self.z.read(video_path)
        container = av.open(io.BytesIO(data))

        frames = []
        for frame in container.decode(video=0):
            img = frame.to_ndarray(format="rgb24")
            img = cv2.resize(img, self.resize)
            frames.append(img)

        if len(frames) >= self.frames:
            idx = np.linspace(0, len(frames)-1, self.frames).astype(int)
            frames = [frames[i] for i in idx]
        else:
            while len(frames) < self.frames:
                frames.append(frames[-1])

        return np.stack(frames)

    def compute_flow(self, frames):
        prev = cv2.cvtColor(frames[0], cv2.COLOR_RGB2GRAY)
        flows = []

        for t in range(1, len(frames)):
            nxt = cv2.cvtColor(frames[t], cv2.COLOR_RGB2GRAY)
            flow = cv2.calcOpticalFlowFarneback(
                prev, nxt, None,
                0.5, 3, 15, 3, 5, 1.2, 0
            )
            flows.append(flow)
            prev = nxt

        flow = np.stack(flows)
        flow = np.transpose(flow, (3,0,1,2))

        flow = np.clip(flow, -10, 10) / 10
        return flow.astype(np.float32)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        video_path = row["video_path"]
        label = int(row["label"])

        if self.use_cache and video_path in self.cache:
            flow = self.cache[video_path]
        else:
            frames = self.read_frames(video_path)
            flow = self.compute_flow(frames)
            if self.use_cache:
                self.cache[video_path] = flow

        x = torch.from_numpy(flow)
        y = torch.tensor(label)

        return x, y

In [ ]:
from torch.utils.data import DataLoader

train_ds = ASLCitizenFlowDataset(train_df, ZIP_PATH, use_cache=True)
val_ds   = ASLCitizenFlowDataset(val_df, ZIP_PATH, use_cache=True)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)


In [ ]:
import torchvision.models.video as video_models
import torch.nn as nn

def build_flow_r3d18(num_classes):
    model = video_models.r3d_18(weights=None)

    old_conv = model.stem[0]
    model.stem[0] = nn.Conv3d(
        in_channels=2,
        out_channels=old_conv.out_channels,
        kernel_size=old_conv.kernel_size,
        stride=old_conv.stride,
        padding=old_conv.padding,
        bias=False
    )

    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = build_flow_r3d18(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

def run_epoch(loader, train=True):
    model.train(train)
    total_loss, total_correct, total = 0.0, 0, 0

    for x, y in tqdm(loader):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        if train:
            optimizer.zero_grad()

        out = model(x)
        loss = criterion(out, y)

        if train:
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * x.size(0)
        total_correct += (out.argmax(1) == y).sum().item()
        total += x.size(0)

    return total_loss/total, total_correct/total


In [ ]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EPOCHS = 3
CHECKPOINT_PATH = "/content/drive/MyDrive/flow_checkpoint.pt"


In [ ]:
model = build_flow_r3d18(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-4
)


In [ ]:
start_epoch = 0

if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
    model.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    start_epoch = checkpoint["epoch"]
    print(f" Resumed training from epoch {start_epoch}")
else:
    print(" Starting training from scratch")


 Starting training from scratch


In [ ]:
def run_epoch(loader, train=True):
    model.train(train)

    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    pbar = tqdm(loader)

    for x, y in pbar:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        if train:
            optimizer.zero_grad()

        outputs = model(x)
        loss = criterion(outputs, y)

        if train:
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * x.size(0)
        total_correct += (outputs.argmax(1) == y).sum().item()
        total_samples += x.size(0)

        pbar.set_postfix(
            loss=total_loss / total_samples,
            acc=total_correct / total_samples
        )

    return (
        total_loss / total_samples,
        total_correct / total_samples
    )


In [ ]:
for epoch in range(start_epoch, EPOCHS):
    print(f"\n==============================")
    print(f"Epoch {epoch + 1} / {EPOCHS}")
    print(f"==============================")

    train_loss, train_acc = run_epoch(train_loader, train=True)
    val_loss, val_acc = run_epoch(val_loader, train=False)

    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f" Val  Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    #   Checkpoint
    torch.save({
        "epoch": epoch + 1,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict()
    }, CHECKPOINT_PATH)

    print("Checkpoint saved")



Epoch 1 / 3


  1%|          | 140/20077 [03:38<7:27:43,  1.35s/it, acc=0, loss=8.12]

# **from scratch model**

In [25]:
import torch
import torch.nn as nn

class Simple3DCNN(nn.Module):
    def __init__(self, num_classes):
        super(Simple3DCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv3d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d((1,2,2)),

            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),

            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
        )

        self.pool = nn.AdaptiveAvgPool3d((1,1,1))
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1, 3, 4)
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [51]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_ds,
    batch_size=4,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_ds,
    batch_size=4,
    shuffle=False
)

In [52]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = train_df['Gloss'].nunique()

model = Simple3DCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [41]:
from tqdm.notebook import tqdm

def train_one_epoch(model, loader):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="Training")

    for videos, labels in pbar:
        videos = videos.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(videos)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix({'loss': total_loss / (pbar.n + 1), 'acc': correct / total})

    return total_loss / len(loader), correct / total

In [42]:
from tqdm.notebook import tqdm

def validate(model, loader):
    model.eval()
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="Validation")

    with torch.no_grad():
        for videos, labels in pbar:
            videos = videos.to(device)
            labels = labels.to(device)

            outputs = model(videos)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

            pbar.set_postfix({'acc': correct / total})

    return correct / total

In [50]:
for epoch in range(3):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_acc = validate(model, val_loader)

    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Acc: {val_acc:.4f}")


Training:   0%|          | 0/20077 [00:00<?, ?it/s]

error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipython-input-3140012475.py", line 19, in __getitem__
    frames = self.loader.read_video(video_path, target_frames=self.frames)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-415612049.py", line 13, in read_video
    data = self.z.read(video_path)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/zipfile/__init__.py", line 1585, in read
    return fp.read()
           ^^^^^^^^^
  File "/usr/lib/python3.12/zipfile